In [ ]:
# Section 1: Training a Simple Model

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Load and prepare the dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# One-hot encode the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = Sequential([
    Dense(10, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(10, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Accuracy: {accuracy:.4f}')

# Save the model in TensorFlow format
tf_model_path = './output/iris_model_tf'
model.save(tf_model_path)
print(f'TensorFlow model saved to {tf_model_path}')


In [ ]:
# Section 2: Conversion to ONNX

import tf2onnx
import onnx

# Define the model path
onnx_model_path = './output/iris_model.onnx'

# Convert the TensorFlow model to ONNX
spec = (tf.TensorSpec((None, X_train.shape[1]), tf.float32, name="input"),)
output_path = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13, output_path=onnx_model_path)

print(f'Model saved to {onnx_model_path}')

Model saved to ./output/iris_model.onnx

In [ ]:
# Section 3: Inference

import onnxruntime as ort
import numpy as np

# Load the ONNX model
onnx_model = ort.InferenceSession(onnx_model_path)

# Prepare the input data for inference
def prepare_input(data):
    return {onnx_model.get_inputs()[0].name: data.astype(np.float32)}

# Make predictions
input_data = prepare_input(X_test)
predictions = onnx_model.run(None, input_data)

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions[0], axis=1)
true_labels = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = np.mean(predicted_labels == true_labels)
print(f'Inference Accuracy: {accuracy:.4f}')

Inference Accuracy: 0.9667

In [ ]:
# Section 3: Inference

import onnxruntime as ort
import numpy as np

# Load the ONNX model
onnx_model = ort.InferenceSession(onnx_model_path)

# Prepare the input data for inference
data = np.array([[4.5, 4.9, 5.1, 5.4],
                 [1.5, 2.9, 3.1, 1.4],
                 [7.5, 6.9, 8.1, 6.4]])

def prepare_input(data):
    return {onnx_model.get_inputs()[0].name: data.astype(np.float32)}

# Make predictions
input_data = prepare_input(data)
predictions = onnx_model.run(None, input_data)

# Convert the predictions to class labels
predicted_labels = np.argmax(predictions[0], axis=1)
print(f'Predicted labels: {predicted_labels}')